In [1]:
import os
import boto3
from botocore.exceptions import NoCredentialsError
from dotenv import load_dotenv

In [2]:
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [4]:
def get_s3_client():                                         # Definiujemy funkcję, która stworzy nam obiekt do komunikacji z chmurą.
    """Inicjalizuje klienta kompatybilnego z S3 dla DigitalOcean."""
    session = boto3.session.Session()                        # Tworzymy nową sesję boto3, która zarządza konfiguracją i stanem połączenia.
    return session.client(                                   # Zwracamy obiekt typu 'client', który posiada metody do przesyłania/pobierania plików.
        's3',                                                # Określamy protokół: używamy standardu S3, który DigitalOcean w pełni obsługuje.
        region_name='fra1',                                  # Podajemy region geograficzny centrum danych (fra1 = Frankfurt).
        endpoint_url='https://fra1.digitaloceanspaces.com',  # Wskazujemy dokładny adres URL serwerów DigitalOcean (zamiast domyślnego AWS).
        aws_access_key_id=os.getenv('DO_SPACES_KEY'),        # Pobieramy z systemu (z pliku .env) Twój publiczny identyfikator dostępu.
        aws_secret_access_key=os.getenv('DO_SPACES_SECRET')  # Pobieramy z systemu Twój tajny klucz (hasło), zachowując bezpieczeństwo.
    )                                                        # Zamykamy nawias wywołania metody client.

In [5]:
def upload_to_spaces(file_name, bucket, object_name=None):   # Definiujemy funkcję z argumentami: nazwa pliku lokalnego, nazwa Space i opcjonalna nazwa w chmurze.
    """Przesyła plik do wskazanego Space."""                 # Krótka dokumentacja wyjaśniająca zadanie funkcji.
    client = get_s3_client()                                 # Wywołujemy naszą poprzednią funkcję, aby uzyskać połączenie (klienta) z DigitalOcean.
    if object_name is None:                                  # Sprawdzamy, czy użytkownik podał nazwę docelową dla pliku w chmurze.
        object_name = file_name                              # Jeśli nie podano nazwy docelowej, używamy nazwy pliku z dysku lokalnego.
    
    try:                                                     # Rozpoczynamy blok 'try', aby przechwycić ewentualne błędy i nie "wywalić" całego programu.
        client.upload_file(file_name, bucket, object_name)   # Kluczowa metoda boto3, która fizycznie wysyła strumień danych z dysku do chmury.
        print(f"✅ Plik {file_name} wylądował w {bucket}/{object_name}") # Informujemy użytkownika o sukcesie operacji.
    except FileNotFoundError:                                # Jeśli pliku nie ma na dysku (np. literówka w nazwie), program wejdzie tutaj.
        print(f"❌ Nie znaleziono pliku: {file_name}")        # Wyświetlamy czytelny komunikat o braku pliku lokalnego.
    except NoCredentialsError:                               # Jeśli klucze w .env są błędne lub ich nie ma, biblioteka wyrzuci ten błąd.
        print("❌ Błąd poświadczeń. Sprawdź plik .env")      # Informujemy o problemie z autoryzacją.

if __name__ == "__main__":                                   # Warunek sprawdzający, czy skrypt jest uruchamiany bezpośrednio (a nie importowany jako moduł).
    SPACE_NAME = "mw-storage-marcin"                         # Definiujemy zmienną z nazwą Twojego kontenera na dane w DigitalOcean.
    
    files_to_upload = [                                      # Tworzymy listę plików, które chcemy umieścić w chmurze.
        "halfmarathon_wroclaw_2024__final.csv",              # Nazwa pierwszego pliku CSV z danymi.
        "halfmarathon_wroclaw_2023__final.csv"               # Nazwa drugiego pliku CSV z danymi.
    ]
    
    for file in files_to_upload:                             # Pętla 'for' przechodzi przez każdy plik wymieniony na powyższej liście.
        upload_to_spaces(file, SPACE_NAME, f"data/raw/{file}") # Wywołujemy funkcję wysyłania, tworząc w chmurze strukturę folderów 'data/raw/'.

✅ Plik halfmarathon_wroclaw_2024__final.csv wylądował w mw-storage-marcin/data/raw/halfmarathon_wroclaw_2024__final.csv
✅ Plik halfmarathon_wroclaw_2023__final.csv wylądował w mw-storage-marcin/data/raw/halfmarathon_wroclaw_2023__final.csv
